Services überwachen
----------

Damit eigene (Micro)Services überwacht werden können, müssen diese Metrics Informationen liefern.

Dazu müssen die (Micro)Services Metrik Informationen liefern, wie der [Auto Shop GmbH](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v3.0.0).

- - -

**Beispiel aus [order_service.py](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v3.0.0/order)**

Metriken definieren

    REQUEST_COUNT = Counter('http_requests_total', 'Total HTTP Requests', ['method', 'endpoint'])
    REQUEST_LATENCY = Histogram('http_request_latency_seconds', 'HTTP request latency', ['method', 'endpoint'])
    MEMORY_USAGE = Gauge('process_memory_usage_bytes', 'Memory usage of the process')
    
Metriken aktualisieren, z.B. bei jedem Aufruf der Webseite

    @app.route('/order')
    def order_page():
    ...
    REQUEST_COUNT.labels(method="GET", endpoint="/order").inc()
    REQUEST_LATENCY.labels(method="GET", endpoint="/order").observe(time.time() - start_time)
    update_memory_usage()
    ..
    
Metriken Prometheus zur Verfügung stellen

    @app.route('/metrics')
    def metrics():
        return generate_latest(), 200, {'Content-Type': CONTENT_TYPE_LATEST}


In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-deployment.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-service.yaml

echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

Für die nächsten Befehle müssen alle Container gestartet sein:

In [ ]:
! kubectl get all,servicemonitors --namespace ms-rest -l tier=microservice

Zu Testzwecken zeigen wir die Metrik Daten des Order Microserivces an.

Damit mehr Daten angezeigt werden (z.B. die Anzahl HTTP Requests), vorher das UI des Webshop bzw. von Order anwählen.


In [ ]:
%%bash
curl -s "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop/order/metrics

- - -
### Prometheus 

Die erstellten Microservices können wir nun mit einer eigenen Prometheus Instanz überwachen.

Dazu brauchen wir die entsprechenden Rechte:   

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-rest
%EOF%


und eine eigene Prometheus Instanz.


In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: NodePort
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  ruleSelector: {}  
  ruleNamespaceSelector:
    matchLabels:
      kubernetes.io/metadata.name: ms-rest
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
  alerting:
    alertmanagers:
      - namespace: ms-rest
        name: alertmanager
        port: web
%EOF%


### ServiceMonitor

Zum Schluss, starten wir das Monitoring mittels `ServiceMonitor`.

Welche Microservices überwacht werden sollen bestimmt der `selector` und welche Ports `endpoints` und `port:`.

Die YAML Dateien der Services sind entsprechend anzupassen:

    apiVersion: v1
    kind: Service
    metadata:
      labels:
        name: order
        tier: microservice
      name: order
    spec:
      ports:
      - port: 8080
        protocol: TCP
        targetPort: 8080
        name: web    
      selector:
        app: order

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
    path: /metrics 
    interval: 30s  
%EOF%


---
### Abfragen von Werten im Prometheus UI

Im User Interface können wir die überwachten Targets und deren Status anschauen.

Dazu ist als Query `up` einzugeben oder via Pulldown -> Status -> Targets die überwachten Microservices anzuzeigen.

Ausserdem können folgende Werte ausgewertet werden:

* http_requests_total{method="GET"} - liefert Anzahl HTTP GET Aufrufe
* process_memory_usage_bytes - den Speicherverbrauch pro Pod

In [ ]:
%%bash
echo "Prometheus UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

Weiter geht es mit dem [Alert Manager](03-alert-servicemesh.ipynb)